In [1]:
import numpy as np

## LM Head Forwording

LM head คือ ตัวแปลงผลลัพธ์ที่ได้จาก transformer block (ที่ให้ output ออกมาเป็น hidden states ของแต่ละตำแหน่งในประโยค) 

Mathemetic definition

$$Logits = h_{last} \cdot W_{vocab} + b_{vocab}$$

Where 
* $h_{last} \in R^{(d_{model}, )}$
* $W_{vocab} \in R^{(d_{model} \times vocab\_size)}$
* $b_{vocab} \in R^{(vocab\_size,)}$ 
* $logits \in R^{(vocab\_size,)}$ 

$h_{last}$ คือ hidden state ของ token ตัวสุดท้ายที่ได้จาก stack ของ transformer 

In [2]:
d_model = 6
vocab_size = 8
V = vocab_size
T = 4


In [3]:
W_vocab = np.random.randn(d_model, V)
b_vocab = np.random.randn(vocab_size)
h = np.random.rand(T, d_model)

print(h)

[[0.02255349 0.16870012 0.5721247  0.9210167  0.58313575 0.76322844]
 [0.19224613 0.54639425 0.29403152 0.82971492 0.70312466 0.85806276]
 [0.62453593 0.5436307  0.4470469  0.19615906 0.36404697 0.82558732]
 [0.41578608 0.4056232  0.46485935 0.20830879 0.1937086  0.65864715]]


In [4]:
logits = h @ W_vocab + b_vocab
print(logits)

[[ 2.23307593e+00 -2.22450795e+00  1.65522988e+00 -3.82662827e-01
  -8.67207793e-01 -1.93948771e+00  5.93770716e-01  1.87317729e+00]
 [ 2.65288192e+00 -2.11610956e+00  1.44889534e+00 -3.70957674e-01
   1.41222341e-03 -1.42262025e+00  1.22104166e+00  3.20280509e+00]
 [ 4.26870232e+00 -1.17305548e+00  8.48799222e-01  5.39467915e-01
  -1.91944824e-01 -1.02898547e+00  8.03814381e-01  2.54856127e+00]
 [ 3.88240535e+00 -1.18742403e+00  1.03592437e+00  8.23262081e-01
  -5.37641843e-01 -1.42152874e+00  5.93735302e-01  2.07642065e+00]]


### In case of many batch

In [5]:
B = 2 # batch size

In [6]:
h = np.random.rand(B, T, d_model)
print (h.shape)

(2, 4, 6)


In [7]:
W_vocab.shape

(6, 8)

In [8]:
h[:, -1, :].shape

(2, 6)

In [9]:
logits = (h[:, -1, :] @ W_vocab) + b_vocab
print("logits => \n")
print(logits)

logits => 

[[ 4.03495665 -1.28564555  0.59113801  0.20045747  0.46734478 -0.86604939
   0.58075805  2.93314083]
 [ 2.92885596 -1.61719677  0.72267276 -1.01193645  0.14097129 -1.40839486
   0.03652673  1.67029531]]


## Back propagation of LM head

from the last chapter (loss And Softmax)

เราทราบว่า

$$\frac {\partial \mathcal {L}} {\partial p} \cdot \frac{\partial p} {\partial Z} = \frac {\partial \mathcal {L}} {\partial Z} \in R^{B \times T \times V}$$

หรือมีมิติเท่ากับ logits

เมื่อ 
* $\mathcal {L}$ คือ loss ของ model
* $p$ คือ ความน่าจะเป็นของแต่ละคำที่จะออกเป็น token ถัดไป 
* $Z$ คือ Logits ที่ได้จากการคำนวณด้วย LM Head

In [10]:
dZ = np.random.rand(B, T, V)
print("dZ =>", dZ.shape)

dZ => (2, 4, 8)


### Start from math
---

#### เมื่อต้องการปรับค่า $W_{vocab}$ เพื่อให้ loss มีค่าลดลงจะต้องปรับโดยการ : 
$$ W^{new}_{vocab} = W^{old}_{vocab} - \alpha \frac{\partial \mathcal {L}} {\partial W_{vocab}}$$

จาก Chain rule : 

$$\frac{\partial \mathcal {L}} {\partial W_{vocab}} = \frac{\partial \mathcal {L}} {\partial p} \cdot \frac{\partial p} {\partial Z} \cdot \frac{\partial Z} {\partial W_{vocab}}$$

จากฟังค์ชัน LM Head สำหรับ linear layer :

$$ Z = hW + b$$

gradient ของ $W$ ต้องเป็น "ผลรวมของ output product":
* ถ้ากรณีของ **last token only** :
$$dW = h_{last}^\intercal dZ$$

* ถ้ากรณีของทุก token :
$$ dW = \sum_{b, t} h_{b, t}^\intercal dZ_{b, t}$$


$$ \frac{\partial Z} {\partial W_{vocab}} = h_{last\ token}^\intercal$$

เมื่อ 
* $h \in \mathbb{R}^{B \times T \times d}$
* $dZ \in \mathbb{R}^{B \times T \times V}$
* $dW \in \mathbb{R}^{d \times V}$

#### เมื่อต้องการปรับค่า $b_{vocab}$ เพื่อให้ loss มีค่าลดลงจะต้องปรับโดยการ : 

$$ b^{new}_{vocab} = b^{old}_{vocab} - \alpha \frac{\partial \mathcal {L}} {\partial b_{vocab}}$$

จาก Chain rule : 

$$\frac{\partial \mathcal {L}} {\partial b_{vocab}} = \frac{\partial \mathcal {L}} {\partial p} \cdot \frac{\partial p} {\partial Z} \cdot \frac{\partial Z} {\partial b}$$

ซึ่ง 

$$ \frac{\partial Z} {\partial b_{vocab}} = 1$$

ได้ว่า

$$ b^{new}_{vocab} = b^{old}_{vocab} - \alpha \frac{\partial \mathcal {L} } {\partial Z}$$

แต่โดยปรกติแล้วเราจะใช้วิธีคำนวณแบบคิดทุก token เนื่องจาก 
* ถ้าคิดเฉพาะ last token &rarr; ได้สัญญาณสอน 1 จุดต่อ 1 sequence ทำให้เรียนช้ามาก ๆ
* คิดทุก token &rarr; ได้สัญญาณสอน ~T-1 จุด ต่อ sequence ซึ่งทำให้ใช้ข้อมูลได้คุ้มค่ากว่า


ตอน train เราทำ teacher forcing : ทุกตำแหน่งใน sequence มี target ของตัวเอง จาก (forward side) &rarr;
* $h \in \mathbb{R} ^ {B \times T \times d}$
* $logits = h\ @\ W_{vocab} + b_{vocab} \in \mathbb{R} ^ {B \times T \times V}$



Backward propagation ของ LM Head :
มีลำดับการไหลย้อนกลับดังนี้

**loss &rarr; logits (dZ) &rarr; W, b &rarr; hidden (dh)**

เมื่อ forward คือ : 

$$Z_{b, t, V} = h_{b, t, d} W + b$$

1. เราต้องมี gradient of logits ก่อน

ถ้า loss เป็น softmax + cross entropy แบบมาตรฐาน เราจะได้ว่า

$$\frac {\partial \mathcal{L}}{\partial Z} = dZ = \frac {P - Y} {N}$$

Where
* $P$ is softmax(logits) $P \in \mathbb {R} ^ {B, T, V}$
* $Y$ is one-hot of target $Y \in \mathbb {R} ^ {B, T, V}$
* $N$ is token count in loss after mask

2. **from $dZ$ to $dW, db$**

$$\frac {\partial \mathcal {L}} {\partial W} = dW = \sum_{b, t} h_{b, t}^\intercal dZ_{b, t}$$

$$\frac {\partial \mathcal {L}} {\partial b} = db = \sum_{b, t} dZ_{b, t}$$

3. **find $dh$**

from chain rule :
$$ \frac {\partial \mathcal {L}} {\partial h} = \frac {\partial \mathcal {L}} {\partial Z} \cdot \frac {\partial Z} {\partial h} = dh$$

$$dh_{b, t} = dZ_{b, t} W^\intercal$$

### Code demo
---

In [11]:
alpha = 0.1


เนื่องจากการดำเนินการ 3 มิติทำได้ยาก วิธีที่น่าจะดีกว่าคือการรวมมิติของ batch และ token เข้าด้วยกัน (กลายเป็น 2 มิติโดยการ)

In [12]:
print("shape of h before reshape: ", h.shape)
h = h.reshape(B * T, d_model)
print("shape of h after reshape: ", h.shape)

shape of h before reshape:  (2, 4, 6)
shape of h after reshape:  (8, 6)


In [13]:
print("shape of dZ before reshape: ", dZ.shape)
dZ = dZ.reshape(B * T, V)
print("shape of dZ after reshape : ", dZ.shape)

shape of dZ before reshape:  (2, 4, 8)
shape of dZ after reshape :  (8, 8)


In [14]:
dW = h.T @ dZ
print("now shape of dW will be (d, V) => ", dW.shape)
print("must be equal to shape of W_vocab =>", W_vocab.shape)

now shape of dW will be (d, V) =>  (6, 8)
must be equal to shape of W_vocab => (6, 8)


In [15]:
db = np.sum(dZ, axis=0)
print("now shape of db will be (V, ) => ", db.shape)
print("same as dW, db shape must be the same as b_vocab", b_vocab.shape)

now shape of db will be (V, ) =>  (8,)
same as dW, db shape must be the same as b_vocab (8,)


In [16]:
dh = dZ @ W_vocab.T
print("current shape of dh will be (B * T, d_model) => ", dh.shape)

dh = dh.reshape(B, T, d_model)
print("then reshape dh back to (B, T, d_model) => ", dh.shape)

current shape of dh will be (B * T, d_model) =>  (8, 6)
then reshape dh back to (B, T, d_model) =>  (2, 4, 6)


In [17]:
dh

array([[[ 1.95853265,  1.10443924, -3.04693362, -0.26610704,
         -0.31023556, -0.30108329],
        [ 3.55686882,  1.38575228, -2.85537898, -1.64670678,
          0.13342577, -0.5915275 ],
        [ 0.9509311 ,  1.0924749 , -2.42161958, -0.94539325,
          0.20472328, -0.19089009],
        [ 1.27632495,  2.58064706, -2.97623434, -0.89613552,
          1.75869084, -0.87011999]],

       [[ 1.60844782,  0.37729073, -1.13260387, -1.21181377,
          0.13685306, -0.14381364],
        [ 0.48971095,  1.80850004, -2.27081742, -0.33401977,
          0.69450736, -0.5110378 ],
        [ 2.99063461,  3.51900504, -5.18162687, -1.46451647,
          0.80482168, -1.04048242],
        [ 2.38676946,  1.16417956, -2.56951516, -1.2277257 ,
         -0.31809368,  0.19951672]]])

In [24]:
h = h.reshape(B, T, d_model)
print(h.shape)

(2, 4, 6)


In [26]:
print(h)

[[[0.11257476 0.62819814 0.52878121 0.80597715 0.15245238 0.14057607]
  [0.73198895 0.94505398 0.39113548 0.01286054 0.11457224 0.01962315]
  [0.39833523 0.53662388 0.2993796  0.05548486 0.71477461 0.79921448]
  [0.60614947 0.94039149 0.53742216 0.52730714 0.1272071  0.59347637]]

 [[0.2754078  0.42117735 0.77628324 0.807218   0.13853213 0.99038687]
  [0.9296552  0.82687484 0.98011043 0.53578386 0.81410852 0.36888141]
  [0.16263207 0.08732671 0.28251458 0.5393773  0.64891455 0.60811781]
  [0.35100815 0.62788596 0.96024176 0.98771624 0.29146622 0.32310992]]]
